# Random Walks en Multiplex de Aerolíneas

## Directorio

In [1]:
#Establecer directorio
cd("$(homedir())/Documents/UNAM/Tesis_Lic")

## Paquetes

Se importan todos los paquetes que se necesitarán para las funciones y visualizaciones del proyecto.

In [2]:
using StatsBase

#using NamedArrays
#using CSV
#using DataFrames

In [3]:
using DelimitedFiles

In [4]:
using RandomNumbers.Xorshifts

In [ ]:
#using LightGraphs

In [ ]:
#using Plots

In [ ]:
#using GraphPlot

In [ ]:
#Leo archivo de diccionario final
#archivo_dicc = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/diccionarios/dictionary_final.csv", ',') 

## Funciones

### Importar datos

In [5]:
info_aerolineas = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/nueva_lista_aerolineas.csv", ',', String)
lista_aerolineas = info_aerolineas[:, 2]

526-element Array{String,1}:
 "Airlines"     
 "global"       
 "Star_Alliance"
 "Sky_Team"     
 "oneworld"     
 "AA"           
 "TK"           
 "DL"           
 "MU"           
 "CZ"           
 "UA"           
 "FR"           
 "BA"           
 ⋮              
 "H7"           
 "GY"           
 "EW"           
 "D6"           
 "7E"           
 "6Y"           
 "6I"           
 "5M"           
 "5G"           
 "4K"           
 "4D"           
 "4B"           

In [6]:
i = 517
airline_code = lista_aerolineas[i]

"EW"

In [7]:
aerolinea_matriz = string(airline_code, "_matriz_du.csv")

"EW_matriz_du.csv"

In [8]:
archivo_dicc = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/diccionarios/dictionary_final.csv", ',')

3073×527 Array{Any,2}:
     ""  "global_airports"      "global_num"  …  "4K_num"  "4D_num"  "4B_num"
    1    "AER"                 1                 "NA"      "NA"      "NA"    
    2    "ASF"                 2                 "NA"      "NA"      "NA"    
    3    "CEK"                 3                 "NA"      "NA"      "NA"    
    4    "DME"                 4                 "NA"      "NA"      "NA"    
    5    "EGO"                 5              …  "NA"      "NA"      "NA"    
    6    "GYD"                 6                 "NA"      "NA"      "NA"    
    7    "KGD"                 7                 "NA"      "NA"      "NA"    
    8    "KZN"                 8                 "NA"      "NA"      "NA"    
    9    "LED"                 9                 "NA"      "NA"      "NA"    
   10    "MRV"                10              …  "NA"      "NA"      "NA"    
   11    "NBC"                11                 "NA"      "NA"      "NA"    
   12    "NJC"                12         

In [9]:
"""
    import_matrix(arch::AbstractString)

Función que importa el archivo de la matriz de adyacencia de R.
"""
function import_matrix(arch::AbstractString)
    archivo_R = readdlm(arch, ',') 
    archivo_R = archivo_R[2:end, :]
    matriz = archivo_R[:, 2:end]
    nombre = convert(Array{Int64,2}, matriz)
    return nombre
end

import_matrix

In [10]:
multiplex = @time import_matrix("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/$aerolinea_matriz")

  0.263374 seconds (389.99 k allocations: 18.982 MiB)


2×2 Array{Int64,2}:
 0  1
 0  0

### Nombres de datos

In [11]:
"""
    airport_names(arch::AbstractString)

Función que obtiene un arreglo con los nombres de aeropuertos de la matriz
"""
function airport_names(arch::AbstractString)
    archivo_R = readdlm(arch, ',', String)
    columna = archivo_R[1, :]
    nombres = columna[2:end]
    return nombres
end

airport_names

In [12]:
data_names_multiplex = @time airport_names("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/$aerolinea_matriz")

  0.129349 seconds (413.05 k allocations: 20.391 MiB)


2-element Array{String,1}:
 "HAM"
 "AMS"

In [ ]:
#data_names_multiplex[65]

### Grado

### Vecinos

In [13]:
"""
    neighbours(M::Array{Int64,2})

Función que obtiene un conjunto de arreglos con los vecinos de cada nodo.
"""
function neighbours(M::Array{Int64,2})
    
    nodos = size(M, 1) #aeropuertos
    vecindades = []
    
    for i in 1:nodos
        vecinos = findall(x -> x != 0, M[i, :])
        push!(vecindades, vecinos)
    end
    vecindades
end

neighbours

In [14]:
neighbors_multiplex = @time neighbours(multiplex);

  0.083112 seconds (204.37 k allocations: 9.820 MiB)


### Random walks

In [15]:
"""

    r_walks(M::Array{Int64,2}, f::Int64, steps::Int64, iter::Int64)

La función r_walks tiene como parámetros:
M, matriz de adyacencia de tipo Array{Int64,2}
f, nodo inicial de tipo Int64
steps, número de pasos de tipo Int64
iter, número de iteraciones de tipo Int64.
La función crea un caminante aleatorio sobre la matriz, el cual tiene igual probabilidad de avanzar 
hacia cualquiera de sus vecinos.
Regresa un arreglo que contiene todas las trayectorias que tomó el caminante aleatorio.
"""
function r_walks(M::Array{Int64,2}, f::Int64, steps::Int64, iter::Int64)
    
    paths = zeros(Int64, (iter, steps + 1))
    s = f
    seed = 1
    vecindades = neighbours(M)

    for i in 1:iter
        paths[i, 1] = f
        
        for p in 1:steps 
            vecindad = vecindades[s]
            
            if vecindad == []
                break
            else
                a = sample(Xoroshiro128Plus(seed), vecindad) 
                paths[i, p + 1] = a
                s = a
                seed += 1
            end
            
        end
        s = f
        
    end
    
    return paths 
end

r_walks

In [16]:
"""

    random_walk(M::Array{Int64,2}, steps::Int64, iter::Int64)

La función random_walk tiene como parámetros:
M, matriz de adyacencia de tipo Array{Int64,2}
steps, número de pasos de tipo Int64
iter, número de iteraciones de tipo Int64.
La función crea un caminante aleatorio sobre la matriz con igual probabilidad de avanzar 
hacia cualquiera de sus vecinos. Esta función recorre todos los nodos de la red como
nodos iniciales y sobre ellos itera para obtener el número de caminos que se le pidió.
Regresa un arreglo que contiene arreglos asociados a todas las trayectorias que tomó el 
caminante aleatorio para cada nodo inicial de la red.
"""
function random_walk(M::Array{Int64,2}, steps::Int64, iter::Int64) 
    
    random_walks = []
    nodos = size(M, 1)
    
    for f in 1:nodos #aeropuertos
        paths = r_walks(M, f, steps, iter)
        push!(random_walks, paths)
    end
    
    random_walks
end

random_walk

In [17]:
random_walk_multiplex = @time random_walk(multiplex, 50, 2)

  0.472936 seconds (444.51 k allocations: 22.707 MiB)


2-element Array{Any,1}:
 [1 2 … 0 0; 1 2 … 0 0]
 [2 0 … 0 0; 2 0 … 0 0]

In [28]:
random_walk_multiplex[2]

2×51 Array{Int64,2}:
 2  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 2  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

### Nodos/aeropuertos visitados

In [19]:
"""
    visited_vertices(W::Array{Any,1})

La función caminos tiene como parámetro W que es un arreglo de arreglos.
Regresa los nodos diferentes a los que llega el caminante.
"""
function visited_vertices(W::Array{Any,1})
    
    cobert = Array[]
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    t = size(W[1], 2) #número de pasos, no necesario
    #visit = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    visit = Array[]
    
    for i in 1:v        
        for j in 1:s
            u = unique(W[i][j, :])
            push!(visit, u)
        end
        
        push!(cobert, visit)
        visit = []
    end
    
    return cobert
end

visited_vertices

In [29]:
visitados = visited_vertices(random_walk_multiplex)

2-element Array{Array,1}:
 Array[[1, 2, 0], [1, 2, 0]]
 Any[[2, 0], [2, 0]]        

In [35]:
visitados =[[1,2,3],[1,2,3]],[[2,0], [2,1]]

(Array{Int64,1}[[1, 2, 3], [1, 2, 3]], Array{Int64,1}[[2, 0], [2, 1]])

In [30]:
"""
    coverage(W::Array{Any,1})

La función cobertura tiene como parámetro W que es un arreglo de arreglos.
Regresa el número de nodos diferentes a los que llega el caminante.
"""
function coverage(W::Array{Any,1})

    cobert = []

    v = length(W) #tamaño de la matriz, número de nodos
    x = visited_vertices(W)
    s = size(x[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    longz = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones

    for i in 1:v #va por los nodos
        for j in 1:s #por las iteraciones
            a = x[i][j]
            deleteat!(a, findall(k->k==0, a))
            l = length(a) #
            longz[j] = l
            
        end

        push!(cobert, longz)
        longz = zeros(Int64, s)
    end

    return cobert
end


coverage

In [31]:
coverage(random_walk_multiplex)

2-element Array{Any,1}:
 [2, 2]
 [1, 1]

In [23]:
"""
    vertex_coverage(W::Array{Any,1})

Da el promedio de cobertura la caminata por nodo inicial.
"""
function vertex_coverage(W::Array{Any,1}) #Esta es la importante

    longitud = Array{Float64}(undef, 0)
    x = coverage(W)

    for i in 1:length(x)
        l = mean(x[i])
        push!(longitud, l)
    end

    longitud
end


vertex_coverage

In [26]:
vertex_coverage(random_walk_multiplex)

2-element Array{Float64,1}:
 3.0
 2.0

In [24]:
"""
    layer_coverage(W::Array{Any, 1})

Da el promedio de la longitud de las caminatas para todos los nodos, como un número.
"""
function layer_coverage(W::Array{Any, 1})
    x = vertex_coverage(W)
    promedio = mean(x)
    return promedio
end


layer_coverage

In [27]:
layer_coverage(random_walk_multiplex)

2.5

### Random walks con nodos globales

In [ ]:
"""
    walk_global(W::Array{Any,1}, airline_code::String, v::Int64)

Función que cambia las caminatas del número de nodo asociado a la aerolínea a número de nodo GLOBAL de aeropuertos.
Parámetros:
W = caminatas aleatorias (arreglo de matrices)
airline_code = código de la aerolínea
v = nodo inicial
"""
function walk_global(W::Array{Any,1}, airline_code::String, v::Int64)
    
    #Ahora se usa la función para obtener los nodos diferentes
    nodos_diferentes = visited_vertices(W)
    
    #vv2b[1][1,:] #Nodos únicos visitados en la iteración 1 con nodo inicial 1 #Leo los visited vertices del archivo
    m = size(W[1], 1) #renglones/iteraciones
    #@show m
    n = size(W[1], 2) #columnas/pasos+1/nodos
    #@show n
    l = length(W) #número de nodos iniciales
    #@show l
    
    #Se lee el diccionario
     #archivo_dicc
    #@show archivo_dicc
    aerol_col = string(airline_code, "_num") #Nombre de la columna
    #@show aerol_col
    coord_aerol = findall(x -> x == aerol_col, archivo_dicc) #Busco las coordenadas de la columna
    #@show coord_aerol
    col_aerol = coord_aerol[1][2] #Segunda coordenada de la posición (columna)
    columna_aerol = archivo_dicc[:, col_aerol] #Obtengo la columna con el nombre del aeropuerto al que corresponde
    #@show columna_aerol
    walk_empty = zeros(Int64, m, n)#Arreglo vacío para nuevos números
        #@show walk_empty
    
    for k = v #1:l #nodo inicial v
        #@show k
        arreglo_visitas = nodos_diferentes[k]
        #@show arreglo_visitas
        caminatas_nodo = W[k] #caminata y sus iteraciones con el mismo nodo inicial
        #@show caminatas_nodo
        
        
        for i in 1:m #voy por las iteraciones
            #@show i
            nodos_dif = arreglo_visitas[i, :] #array/caminata única de los nodos que buscará
            #@show nodos_dif
            
            for j in 1:length(nodos_dif[]) #vamos por los nodos diferentes por los que pasé
                #@show j
                
                nodo_original = nodos_dif[][j] #index original de la aerolínea
                #@show nodo_original
                
                #Busco la posición original del aeropuerto que se asignará
               # if 
                posicion_original = findall(x -> x == nodo_original, archivo_dicc[:, col_aerol]) 
                #Este arreglo tendrá la coordenada de la posición del nodo inicial, sin embargo, si llegué a un endpoint, 
                #buscará la coordenada de la posición del nodo 0 y te regresará un un arreglo vacío.
                #@show posicion_original
                coords_ys = findall(x -> x == nodo_original, caminatas_nodo[i, :]) #Encuentra uno de los nodos dif en la iteración
                        #@show coords_ys
                
                #Se busca el nombre del aeropuerto
                
                for q in 1:length(coords_ys) #Se itera sobre las posiciones en las caminatas
                        #@show length(coords_ys)
                        #@show q
                        
                        pos_actual = coords_ys[q]
                
                        if isempty(posicion_original) == false
                    
                            numero_asignado = archivo_dicc[posicion_original, 3]
                            #@show numero_asignado
                            #Busco el primer nodo diferente en la iteración i
                            
                            
                            walk_empty[i, pos_actual] = numero_asignado[] #asigno valor de diccionario al arreglo vacío i, pos_actual
                            #@show walk_empty
                    
                
                
                        else break
                    end
                
                end
                
            end
            #@show walk_empty

        end
        #@show walk_empty
        
        
    end
    
    return walk_empty
   
end

In [ ]:
for v = 1
    walk_global_multiplex = @time walk_global(random_walk_multiplex, airline_code, v)
    return walk_global_multiplex
end

In [ ]:
function walks_global(W::Array{Any,1}, airline_code::String)
    
    t = length(W)
    #@show t
    
    arreglo = Array[]
    
    for v in 267:t
        #@show v
        #archivo_caminata_global = string(airline_code, "_", v,"_walk_global.csv")
        #@show archivo_caminata_global
        
        walk_global_multiplex = walk_global(W, airline_code, v)
        #return walk_global_multiplex
        push!(arreglo, walk_global_multiplex)
        #writedlm("/Users/Yuriko/Desktop/carpetita/$archivo_caminata_global", walk_global_multiplex, ',')
    end
    return arreglo
end

In [ ]:
#walks_global(random_walk_multiplex, airline_code)

In [ ]:
#for v = 1:length(random_walk_multiplex)
#    @show v
#    archivo_caminata_global = string(airline_code, "_", v,"_walk_global.csv")
#    @show archivo_caminata_global
#	walk_global_multiplex = @time walk_global(random_walk_multiplex, airline_code, v)
#    return walk_global_multiplex
#    writedlm("/Users/Yuriko/Desktop/carpetita/$archivo_caminata_global", walk_global_multiplex, ',')
#end

### Random walks con nombres

In [ ]:
"""
    walk_names(W::Array{Any,1}, airline_code::String, v::Int64)

Función que cambia las caminatas del número de nodo asociado a la aerolínea a nombres de aeropuertos.
Parámetros:
W = caminatas aleatorias (arreglo de matrices)
airline_code = código de la aerolínea
v = nodo inicial
"""
function walk_names(W::Array{Any,1}, airline_code::String, v::Int64, dicc) 
    
    #Ahora se usa la función para obtener los nodos diferentes
    nodos_diferentes = visited_vertices(W)
    m = size(W[1], 1) #renglones
    n = size(W[1], 2) #columnas
    l = length(W) #número de nodos iniciales
    #ep = ["Endpoint"]
    
    #Se lee el diccionario
    #archivo_dicc = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/diccionarios/dictionary_final.csv", ',') 
    archivo_dicc = dicc
    
    aerol_col = string(airline_code, "_num") #Nombre de la columna
    coord_aerol = findall(x -> x == aerol_col, archivo_dicc) #Busco las coordenadas de la columna
    col_aerol = coord_aerol[1][2] #Segunda coordenada de la posición (columna)
    columna_aerol = archivo_dicc[:, col_aerol] #Obtengo la columna con el nombre del aeropuerto al que corresponde
    walk_nothing = Array{Any}(undef, m, n)#Array{Union{Nothing, String}}(nothing, m, n) #archivo vacío de una matriz para un nodo inicial
    #@show walk_nothing
    
    for k = v #nodo inicial v
        arreglo_visitas = nodos_diferentes[k]
        caminatas_nodo = W[k]
        
        for i in 1:m #voy por las iteraciones
            nodos_dif = arreglo_visitas[i, :] #array/caminata única de los nodos que buscará
            
            for j in 1:length(nodos_dif[]) #para cada uno de los nodos diferentes visitados
                nodo_original = nodos_dif[][j] #tomo el nodo original visitado
                
                #Busco la posición original del aeropuerto visitado en la columna de la aerolínea
                posicion_original = findall(x -> x == nodo_original, archivo_dicc[:, col_aerol])
                
                #Ahora busco dónde se colocará el nombre en las posiciones correctas del arreglo vacío
                #Busco el primer nodo diferente en la iteración i
                #Encuentra todas las coordenadas de los nodos dif en la iteración original
                coords_ys = findall(x -> x == nodo_original, caminatas_nodo[i, :]) 
            
                
                for q in 1:length(coords_ys) #Se itera sobre las posiciones en las caminatas
                    
                    pos_actual = coords_ys[q]
                
                    if isempty(posicion_original) == false
                        
                        #Se busca el nombre del aeropuerto
                        nombre_asignado = archivo_dicc[posicion_original, 2] #y=2 porque esa es la columna de los nombres
                    
                        walk_nothing[i, pos_actual] = nombre_asignado[] #asigno valor de diccionario al arreglo vacío i, pos_actual

                        else walk_nothing[i, pos_actual] = " " #break
                    end
                end
                
            end

        end
        
    end
    return walk_nothing
end

In [ ]:
for v = 1:2#:length(random_walk_multiplex)
	walk_names_multiplex = @time walk_names(random_walk_multiplex, airline_code, v, archivo_dicc)
    walk_names_multiplex
    archivo_caminata_nombres = string(airline_code, "_", v,"_walk_names.csv")
    writedlm("/Users/yuriko/Desktop/$archivo_caminata_nombres", walk_names_multiplex, ',')
end